In [1]:
# Import necessary libraries
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras.applications.densenet import DenseNet121
from keras.layers import Input
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array

In [2]:
# Load pre-trained MTCNN model
mtcnn_model = MTCNN()

In [3]:
# Load pre-trained DenseNet model
input_shape = (224, 224, 3)
input_tensor = Input(shape=input_shape)
densenet_model = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [4]:
# Load an image for face detection
image_path = 'C:\\Users\\Mehedi Hassan\\Desktop\\file3.jpg'
image = load_img(image_path, target_size=(224, 224))
image_array = img_to_array(image)
image_array = np.expand_dims(image_array, axis=0)

In [5]:
# Detect faces using MTCNN
print(image.size)
image_array1 = np.array(image)
print(image_array1.shape)
faces = mtcnn_model.detect_faces(image_array1)
for face in faces:
    x, y, width, height = face['box']
    cv2.rectangle(image_array[0], (x, y), (x + width, y + height), (0, 255, 0), 2)

(224, 224)
(224, 224, 3)
1/1 [==============================] - 0s 142ms/step


In [6]:
# Extract face regions
face_regions = []
for face in faces:
    x, y, width, height = face['box']
    # Ensure the bounding box is within the bounds of the image
    x = max(0, x)
    y = max(0, y)
    width = min(image_array.shape[1] - x, width)
    height = min(image_array.shape[0] - y, height)
    cv2.rectangle(image_array[0], (x, y), (x + width, y + height), (0, 255, 0), 2)

In [7]:
# Print the shapes and types of the elements in face_regions
for i, face_region in enumerate(face_regions):
    print(f"Face region {i}: shape={face_region.shape}, dtype={face_region.dtype}")

# Print the length of face_regions
print(f"Number of face regions: {len(face_regions)}")

# Convert to float32
face_regions = [face_region.astype('float32') for face_region in face_regions]

# Normalize to 0-1
face_regions = [face_region / 255 for face_region in face_regions]

if face_regions:
    features = densenet_model.predict(np.array(face_regions))
else:
    print("No faces detected.")

Number of face regions: 0
No faces detected.


In [ ]:
# Now you have the features extracted from each detected face using DenseNet!
# You can use these features for face recognition or other tasks.
 
# Display the image with bounding boxes around detected faces
cv2.imshow('Detected Faces', image_array[0])
cv2.waitKey(0)
cv2.destroyAllWindows()